In [23]:
# please commment %%capture in the first line of the follwing code block,
# , if you want to see the output. 
!pip install -q gwpy

In [24]:
%%capture
!pip install openmim
!mim install mmcv-full

In [25]:
%%capture
!mkdir -p ~/.insightface/models
%cd ~/.insightface/models
!wget https://keeper.mpdl.mpg.de/f/2d58b7fed5a74cb5be83/?dl=1 -O antelopev2.zip
!wget https://keeper.mpdl.mpg.de/f/8faabd353cfc457fa5c5/?dl=1 -O buffalo_l.zip
!mkdir -p antelopev2 && cd antelopev2 && unzip -o ../antelopev2.zip
!mkdir -p buffalo_l && cd buffalo_l && unzip -o ../buffalo_l.zip

In [26]:
# !curl -LO https://github.com/open-mmlab/mmcv/archive/refs/tags/v1.6.0.tar.gz
# !tar xzf v1.6.0.tar.gz
# %cd mmcv-1.6.0
# !pip install -r requirements/optional.txt
# !MMCV_WITH_OPS=1 pip install -e . -v
# %cd /content

In [27]:
%%capture
%cd /content
!git clone https://github.com/yhw-yhw/SHOW.git
!wget https://www.dropbox.com/s/gqdcu51ilo44k3i/models.zip?dl=0 -O models.zip
!wget https://www.dropbox.com/s/r14bl9mhvngohla/data.zip?dl=0 -O data.zip
!unzip data.zip
!7za x models.zip

In [28]:
%%capture
%cd /content
!cd /content/SHOW/modules/MICA && pip install -r requirements.txt
!cd /content/SHOW/modules/PIXIE && pip install -r requirements.txt
!cd /content/SHOW/modules/PyMAF && pip install -r requirements.txt
!cd /content/SHOW/modules/DECA && pip install -r requirements.txt
!cd /content/SHOW && pip install -r requirements.txt
!pip uninstall -y xtcocotools && pip install xtcocotools --no-binary xtcocotools

In [29]:
%%capture
%cd /content
!git clone https://github.com/open-mmlab/mmdetection
!cd /content/mmdetection && python setup.py install
!git clone https://github.com/open-mmlab/mmpose
!cd /content/mmpose && python setup.py install

In [3]:
%env mmpose_root = /content/mmpose

env: mmpose_root=/content/mmpose


In [30]:
!rm /usr/local/lib/python3.8/dist-packages/torchgeometry/core/conversions.py
!cp /content/SHOW/conversions.py /usr/local/lib/python3.8/dist-packages/torchgeometry/core/conversions.py

In [31]:
%%capture
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.13.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [32]:
%%capture
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  # download models
  !wget -O /content/openpose/models/hand/pose_iter_102000.caffemodel https://polybox.ethz.ch/index.php/s/Oim76cuqrDVbdxm/download
  !wget -O /content/openpose/models/pose/body_25/pose_iter_584000.caffemodel https://polybox.ethz.ch/index.php/s/m5NQAhd7ukVPRoL/download
  !wget -O /content/openpose/models/face/pose_iter_116000.caffemodel https://polybox.ethz.ch/index.php/s/cEaF1FTpKjjJZbH/download
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. -DUSE_CUDNN=OFF && make -j`nproc`


In [ ]:
!cd /content/SHOW && python main.py --speaker_name -1 --all_top_dir ./test/demo_video/half.mp4

/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
2023-02-24 05:57:24.668226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 05:57:25.683403: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or